In [ ]:
import sys
import os

source_path = os.path.expanduser('source/')
sys.path.append(source_path)

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
from IPython.display import HTML
import matplotlib.animation as animation

%matplotlib inline

In [ ]:
#gregs tools
from infoenginessims.api import *
from infoenginessims.integrators import rkdeterm_eulerstoch
from infoenginessims.dynamics import langevin_underdamped, langevin_overdamped
from infoenginessims import simulation
from infoenginessims.simprocedures import basic_simprocedures as sp
from infoenginessims.simprocedures import running_measurements as rp
from infoenginessims import analysis
import infoenginessims.analysis.running_quantities
import infoenginessims.analysis.hists_1D

In [ ]:
#kyle tools
import kyle_tools as kt
from protocol_designer import potentials, protocol, system
from measure import MeasurementDevice, Measurement, TrajectoryEnsemble

### Totally Explotatory Protocols, no useful protocols implemented yet. Protocol and simulation parameters are not motivated by a particular computational goal
#### comments sparse in this notebook pending further investigation, but it follows the same process as the 1D bit flip notebook

In [ ]:
#here we pull in the potential for the 4d system of two coupled
from fq_systems import RF_RF_pot

eq_sys = system.System(RF_RF_pot.trivial_protocol(), RF_RF_pot)


### These are the physical parameters that characterize the system

In [ ]:
#ballpark params:
kT = .41*1.38E-23
C = 10*400E-12
R = 371
L = 10E-10

#these are some relevant dimensionful scales: alpha are the natural units for the JJ fluxes and U_0 is the natural scale for the potential
alpha = 2.07E-15 / (2*np.pi)
#IMPORTANT: all energies are measured in units of U_0
U_0 = alpha**2 / L

#these are important dimensionless simulation quantities, accounting for 
#m being measured in units of C, lambda in units of 1/R, energy in units of U_0
m_prime = np.array((1, 1, 1/2, 1/2))
lambda_prime = np.array((2, 2, 1, 1))
kT_prime = kT/U_0

print('Some Common Circuit Parameters')
Q = R*sqrt(C/L)
print( 'Q:{:.2f}'.format(Q))
omeg = 1/(2*np.pi*sqrt(L*C))
print('w_LC in GHz:{:.2f}'.format(omeg/1E9))
print('ring down in ns:{:.2f}'.format(1E9*Q/omeg))
j_c = alpha/L
print('critical current density in mu_A:{:.2f}'.format(1E6*j_c))

### Below is a bit of a plyaground for messing with protocols

In [ ]:
params = np.zeros((3,RF_RF_pot.N_params))
params[:] = RF_RF_pot.default_params
params = params.transpose()
params[4,1] = 0
params[0,1] = 4
params[1,1] = 4
#params[[6,7],:] = 5,5,5
#params[[6,7],1] =1,1
#params[8,:] = .05
#params[1,1] = 2.64
t1=[0,1]
t2=[2,3]
p1 = protocol.Protocol(t1, params[:,[0,1]])
p1.interpolation = 'step'
p2 = protocol.Protocol(t2, params[:,[1,2]])
p2.interpolation = 'step'

test_prot = protocol.Compound_Protocol([p1,p2])

test_sys = system.System(test_prot, RF_RF_pot)

### Now we start the actual simulation steps

In [ ]:

system=eq_sys
#system.protocol.normalize()
#system.protocol.time_stretch(1)


In [ ]:
N=1000
initial_state = system.eq_state(N, M=m_prime)


In [ ]:
nbins= 40
phi_hist = np.histogram(initial_state[:,0,0], bins=nbins)

analysis.hists_1D.plot_hist(phi_hist);

In [ ]:

fig, ax = system.show_potential(0, manual_domain=[[-2,-2,0,0],[8,8,0,0]])
fig, ax = system.show_potential(1, manual_domain=[[-2,-2,0,0],[8,8,0,0]])
fig.set_figwidth(10)
fig.set_figheight(10)


In [ ]:
gamma = (lambda_prime/m_prime) * np.sqrt(L*C) / (R*C) 
theta = 1/m_prime
eta = (L/(R**2 * C))**(1/4) * np.sqrt(kT_prime*lambda_prime) / m_prime

    

thermal = 1
temp = 1

gamma = gamma * thermal
eta = gamma * sqrt(thermal)*sqrt(temp)

dynamic = langevin_underdamped.LangevinUnderdamped(theta, gamma, eta,
                                                       system.get_external_force)

In [ ]:
for item in [gamma, theta, eta]:
    print(item)

In [ ]:
integrator = rkdeterm_eulerstoch.RKDetermEulerStoch(dynamic)

In [ ]:
procedures = [
              sp.ReturnFinalState(),
              sp.MeasureAllState(trial_request=slice(0, 1000)),  
              rp.MeasureAllValue(rp.get_dW, 'all_W'),
              rp.MeasureFinalValue(rp.get_dW, 'final_W'),
             ]

In [ ]:
nsteps = 30_000

total_time = 30*(system.protocol.t_f-system.protocol.t_i)



dt = total_time / nsteps
sim = simulation.Simulation(integrator.update_state, procedures, nsteps, dt,
                            initial_state)

sim.system = system

In [ ]:
%%time

sim.output = sim.run(verbose=True)

In [ ]:
all_state = sim.output.all_state['states']
all_W = sim.output.all_W
final_W = sim.output.final_W

In [ ]:
end_plot_time = total_time #* 1 / 100
trial_indices = np.s_[:10]


analysis.running_quantities.plot_running_quantity(all_state[:1000,:,0,0],
                                                  final_time=total_time,
                                                  end_plot_time=end_plot_time, title='zeta v t')

analysis.running_quantities.plot_running_quantity(all_state[:1000,:,1,0],
                                                  final_time=total_time,
                                                  end_plot_time=end_plot_time, title='zeta\' v t')

In [ ]:
#color coding is set to default, 
%%capture
ani_exp = kt.animate_sim(all_state[...,:2,0], total_time, color_by_state=False, key_state=None)
HTML(ani_exp.to_jshtml(fps=20))

In [ ]:
ani_exp

In [ ]:
analysis.running_quantities.plot_running_quantity(all_W[:1000],
                                                  final_time=total_time,
                                                  end_plot_time=total_time)

In [ ]:
kt.animate_hist_1D(all_state, total_time, which_axes=[np.s_[:,:,0,0]], frame_skip=100)

In [ ]:
system.protocol.show_substage_times()

In [ ]:
final_W_hist = np.histogram(final_W, bins=50)
fig, ax = analysis.hists_1D.plot_hist(final_W_hist, log=True)
m=final_W.mean()
s=final_W.std()
print(m)
ax.axvline(m, color='k')
ax.axvline(m-s, color='k')
ax.axvline(m+s, color='k')

In [ ]:
#np.savez( 'qualtalk/work_{}'.format(int(total_time)) , aw1=aw1, aw2=aw2, final_W=final_W, )